In [48]:
import os, sys, json, pyld, glob
from pyld import jsonld
from collections import OrderedDict
import traceback



def ldid (k,value=dict()):
    return {k: value}
    
    
def sortld(data):
    
    '''
    A sorted JSON-LD file with ctx, id, type first, then the rest of the keys in alphabetical order
    '''
    
    from collections import OrderedDict
    
    out = OrderedDict()
    for ld in '@context id type'.split():
        if ld in data:
            out[ld] = data[ld]
    for ld in sorted(data.keys()):
        if ld not in out:
            out[ld] = data[ld]
            
    return out

In [49]:
# exp = glob.glob('experiment/*.json')

# for e in exp:
#     try:
#         with open(e) as f:
            
#             print(e,f)
#             data = json.load(f)
            
#             data['@context'] = data['@context'].replace('data_descriptors/','')
            
#             data['type'] = 'experiment'
            
#             # data['experiment'] = data['experiment-id'].replace('.json','')
#             # now name 
            
#             data['activity'] = [ldid(i['@id'].replace('cmip6plus:activity/','')) for i in data['activity-id']]
            
#             data['sub_experiment'] = ldid(data['sub-experiment-id']['@id'].replace('cmip6plus:experiment/sub-id/',''),{})
            
#             data['parameters']['id'] = 'additional_parameters'
            
#             data['parent_experiment'] = [ldid(i["@id"].split('/')[-1]) if not isinstance(i,str) else ldid('none') for i in data['parent']['experiment-id'] ]
            
#             data['parent_activity'] = [ldid(i['@id'].split('/')[-1]) if not isinstance(i,str) else ldid('none') for i in data['parent']['activity-id']]
            
#             data['model_realms'] = {}
#             for r in data['model-components']['required']:
#                 data['model_realms'][r['@id'].split('/')[-1]] = dict(is_required=True)
                
#             for a in data['model-components']['additional-allowed']:
#                 data['model_realms'][a['@id'].split('/')[-1]] = dict(is_required=False)
                
            
#             del data['activity-id'], data['sub-experiment-id'], data['parent'], data['experiment'], data['model-components']
        
#         data = OrderedDict(sorted(data.items()))
                
#         with open(e,'w') as f:
#             json.dump(data,f,indent=4)
#     except Exception as e:
#         print('-----ERROR----',e, data)
#         traceback.print_exc()

        

In [51]:
# exp = glob.glob('experiment/*.json')

# for e in exp:
#         with open(e) as f:
#             data = json.load(f)
#             data = sortld(data)
                
#         with open(e,'w') as f:
#             json.dump(data,f,indent=4)
